<a href="https://colab.research.google.com/github/joelmaf/fundamentos/blob/main/ALUNO_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definição do Problema: Regressão Linear

* **Regressão**: predição de saídas contínuas, nota obtida com base em horas estudadas

In [ ]:
#!pip install dash

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html


### Coleta dos dados

In [ ]:
# Configurar semente para reprodutibilidade
np.random.seed(42)

# Definir "Horas Estudadas" entre 0 e 100
Horas_Estudadas = 100 * np.random.rand(100, 1)

# Definir "Nota Obtida" como uma função direta de "Horas Estudadas" com ruído
# Exemplo: Nota obtida aumenta proporcionalmente ao aumento de horas estudadas, até o limite de 10
Nota_Obtida = 0.1 * Horas_Estudadas + np.random.normal(0, 1, (100, 1))

# Limitar notas para o intervalo de 0 a 10 (caso o ruído ultrapasse o limite superior)
Nota_Obtida = np.clip(Nota_Obtida, 0, 10)


df = pd.DataFrame({
    "Horas_Estudadas": Horas_Estudadas.flatten(),
    "Nota_Obtida": Nota_Obtida.flatten()
})

### Análise exploratória dos dados

In [ ]:
# Estatísticas Descritivas




#### Limpeza e transformação

In [ ]:
X = df['Horas_Estudadas'].values.reshape(-1, 1)
y = df['Nota_Obtida'].values.reshape(-1, 1)

#### Visualização e análise

In [ ]:
# Histograma de Nota_Obtida
plt.figure(figsize=(14, 6))


plt.title('Distribuição das Notas Obtidas')
plt.xlabel('Nota Obtida')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Scatter plot para mostrar a relação entre Horas Estudadas e Nota Obtida
plt.figure(figsize=(14, 6))


plt.title('Relação entre Horas Estudadas e Nota Obtida')
plt.xlabel('Horas Estudadas')
plt.ylabel('Nota Obtida')
plt.grid(True)
plt.show()

In [ ]:
# Plot de linha para tendências de estudo
dados_ordenados_corrigidos = df.sort_values(by="Horas_Estudadas").reset_index(drop=True)

plt.figure(figsize=(14, 6))
plt.plot(dados_ordenados_corrigidos['Horas_Estudadas'], dados_ordenados_corrigidos['Nota_Obtida'], linestyle='-', marker='o', color='b', alpha=0.7)
plt.title('Tendência entre Horas Estudadas e Nota Obtida')
plt.xlabel('Horas Estudadas')
plt.ylabel('Nota Obtida')
plt.grid(True)
plt.show()

In [ ]:
# Média de Nota Obtida por Faixa de Horas Estudadas

# Definir faixas de horas estudadas e agrupar os dados
intervalos = pd.cut(df['Horas_Estudadas'], bins=[0, 20, 40, 60, 80, 100])
# Calcular a média de Nota Obtida para cada faixa de horas estudadas
dados_por_intervalo = df.groupby(intervalos)['Nota_Obtida'].mean()

plt.figure(figsize=(14, 6))
dados_por_intervalo.plot(kind='bar', edgecolor='black', color='skyblue')
plt.title('Média de Nota Obtida por Faixa de Horas Estudadas')
plt.xlabel('Faixas de Horas Estudadas')
plt.ylabel('Média de Nota Obtida')
plt.show();

# Dashboards

In [ ]:
# Inicializar o app Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard de Visualização de Dados de Estudo e Desempenho"),

    # Gráfico de Barras - Média de Nota Obtida por Faixa de Horas Estudadas
    dcc.Graph(
        id='media-nota-faixa-horas',
        figure=px.bar(dados_por_intervalo, x=dados_por_intervalo.index.astype(str), y="Nota_Obtida",
                      title="Média de Nota Obtida por Faixa de Horas Estudadas",
                      labels={"x": "Faixa de Horas Estudadas", "y": "Média de Nota Obtida"})
    ),

    # Gráfico de Dispersão com Linha de Regressão
    dcc.Graph(
        id='dispersao-horas-nota',
        figure=px.scatter(df, x="Horas_Estudadas", y="Nota_Obtida", trendline="ols",
                          title="Relação entre Horas Estudadas e Nota Obtida")
    ),


    # Gráfico Boxplot para Horas Estudadas e Nota Obtida
    dcc.Graph(
        id='boxplot-horas-nota',
        figure=go.Figure(data=[
            go.Box(y=df['Horas_Estudadas'], name="Horas Estudadas"),
            go.Box(y=df['Nota_Obtida'], name="Nota Obtida")
        ]).update_layout(title="Distribuição de Horas Estudadas e Nota Obtida")
    ),

    # Heatmap de Correlação
    dcc.Graph(
        id='heatmap-correlacao',
        figure=px.imshow(df.corr(), text_auto=True, aspect="auto",
                         title="Mapa de Correlação entre Horas Estudadas e Nota Obtida")
    )
])

# Executar o app
if __name__ == '__main__':
    app.run_server(debug=True)



python app.py
O dashboard estará disponível localmente no navegador (geralmente no endereço http://127.0.0.1:8050)

# Dividir os dados em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Treinar o modelo

In [ ]:
# Ajustar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
novo = [[12]]
y_pred = model.predict(novo)
y_pred